In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import zipfile
import io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("drive/MyDrive/AudiosIa/2025-07.csv")


# Extraer datos

con esto se puede notar que datos se pueden limpiar porque hay datos mal puestos como generos no existentes, edades fuera de rango o estaciones de bicis con numeros con guiones


In [ ]:
display(df.head(50))

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha_Arribo,Hora_Arribo
0,M,30.0,4998803,173,30/06/2025,23:33:59,348,01/07/2025,00:00:11
1,M,27.0,3976669,009,30/06/2025,23:41:24,086,01/07/2025,00:00:21
2,M,43.0,8223240,576,30/06/2025,23:48:16,296,01/07/2025,00:00:24
3,M,26.0,6050157,365,30/06/2025,23:52:12,329,01/07/2025,00:00:27
4,M,23.0,5124706,144,30/06/2025,23:54:55,575,01/07/2025,00:00:46
5,M,19.0,2053122,137,30/06/2025,23:33:00,343,01/07/2025,00:00:52
6,M,48.0,4574182,124,30/06/2025,23:46:14,576,01/07/2025,00:00:58
7,M,28.0,8244592,595,30/06/2025,23:47:40,618,01/07/2025,00:01:16
8,M,39.0,7042187,160,30/06/2025,23:17:37,255,01/07/2025,00:01:28
9,M,48.0,6648281,271-272,30/06/2025,23:49:30,545,01/07/2025,00:01:30


aqui ya puedo ir viendo como si hay registro de personas con edades irreales

In [ ]:
print(f"la edad mas grande registrada es: {df.Edad_Usuario.max()}")

la edad mas grande registrada es: 124.0


investigue cuanto era el limite de tiempo para usar una ecobici y era de 45 min entonces vi que hay registros que tienen tiempos irreales


In [ ]:
df['Retiro_Datetime'] = pd.to_datetime(df['Fecha_Retiro'] + ' ' + df['Hora_Retiro'], format='%d/%m/%Y %H:%M:%S')
df['Arribo_Datetime'] = pd.to_datetime(df['Fecha_Arribo'] + ' ' + df['Hora_Arribo'], format='%d/%m/%Y %H:%M:%S')

df['Duracion_Uso'] = df['Arribo_Datetime'] - df['Retiro_Datetime']

max_duration = df['Duracion_Uso'].max()

display(df[['Fecha_Retiro', 'Hora_Retiro', 'Fecha_Arribo', 'Hora_Arribo', 'Duracion_Uso']].head())
print(f"la maxima duracion es : {max_duration}")

,Fecha_Retiro,Hora_Retiro,Fecha_Arribo,Hora_Arribo,Duracion_Uso
0,30/06/2025,23:33:59,01/07/2025,00:00:11,0 days 00:26:12
1,30/06/2025,23:41:24,01/07/2025,00:00:21,0 days 00:18:57
2,30/06/2025,23:48:16,01/07/2025,00:00:24,0 days 00:12:08
3,30/06/2025,23:52:12,01/07/2025,00:00:27,0 days 00:08:15
4,30/06/2025,23:54:55,01/07/2025,00:00:46,0 days 00:05:51


la maxima duracion es : 637 days 23:09:20


In [ ]:
print("Unique values in Genero_Usuario column:")
print(df['Genero_Usuario'].unique())

Unique values in Genero_Usuario column:
['M' 'F' '?' 'O' nan]


In [ ]:
print("Missing values per column:")
print(df.isnull().sum())

Missing values per column:
Genero_Usuario            3
Edad_Usuario             80
Bici                      0
Ciclo_Estacion_Retiro     0
Fecha_Retiro              0
Hora_Retiro               0
Ciclo_EstacionArribo      0
Fecha_Arribo              0
Hora_Arribo               0
Retiro_Datetime           0
Arribo_Datetime           0
Duracion_Uso              0
dtype: int64


In [ ]:
long_duration_records = df[df['Duracion_Uso'] > pd.Timedelta(minutes=45)].shape[0]

print(f"Number of records with duration greater than 45 minutes: {long_duration_records}")

Number of records with duration greater than 45 minutes: 22887


# **limpieza** de datos

Fechas y horas → columnas datetime

In [ ]:
df["start_dt"] = pd.to_datetime(
    df["Fecha_Retiro"] + " " + df["Hora_Retiro"],
    dayfirst=True, errors="coerce"
)
df["end_dt"] = pd.to_datetime(
    df["Fecha_Arribo"] + " " + df["Hora_Arribo"],
    dayfirst=True, errors="coerce"
)

print("\nEjemplo fechas combinadas:")
print(df[["Fecha_Retiro","Hora_Retiro","start_dt"]].head())



Ejemplo fechas combinadas:
  Fecha_Retiro Hora_Retiro            start_dt
0   30/06/2025    23:33:59 2025-06-30 23:33:59
1   30/06/2025    23:41:24 2025-06-30 23:41:24
2   30/06/2025    23:48:16 2025-06-30 23:48:16
3   30/06/2025    23:52:12 2025-06-30 23:52:12
4   30/06/2025    23:54:55 2025-06-30 23:54:55


Duración de viajes

In [ ]:
df["duration_min"] = (df["end_dt"] - df["start_dt"]).dt.total_seconds() / 60

before = len(df)
df = df[(df["duration_min"] > 0) & (df["duration_min"] <= 45)].copy()
after = len(df)

print(f"\nFilas eliminadas por duración inválida (>45 o <=0): {before - after}")
print("Duración mínima:", df["duration_min"].min(),
      "máxima:", df["duration_min"].max())



Filas eliminadas por duración inválida (>45 o <=0): 22887
Duración mínima: 0.3 máxima: 45.0


Normalización de estaciones

In [ ]:
for c in ["Ciclo_Estacion_Retiro", "Ciclo_EstacionArribo"]:
    df[c] = df[c].astype(str).str.extract(r"(\d+)", expand=False)
    df[c] = df[c].astype("Int64")

print("\nEjemplo estaciones normalizadas:")
print(df[["Ciclo_Estacion_Retiro","Ciclo_EstacionArribo"]].head(10))



Ejemplo estaciones normalizadas:
   Ciclo_Estacion_Retiro  Ciclo_EstacionArribo
0                    173                   348
1                      9                    86
2                    576                   296
3                    365                   329
4                    144                   575
5                    137                   343
6                    124                   576
7                    595                   618
8                    160                   255
9                    271                   545


Validación de edades

In [ ]:
if "Edad_Usuario" in df.columns:
    n_before = df["Edad_Usuario"].isna().sum()
    df.loc[~df["Edad_Usuario"].between(10, 95), "Edad_Usuario"] = np.nan
    n_after = df["Edad_Usuario"].isna().sum()
    print(f"\nEdades corregidas (fuera de rango → NaN): {n_after - n_before}")
    print("Ejemplo edades:", df["Edad_Usuario"].head(10).tolist())


Edades corregidas (fuera de rango → NaN): 36
Ejemplo edades: [30.0, 27.0, 43.0, 26.0, 23.0, 19.0, 48.0, 28.0, 39.0, 48.0]


Limpieza de género

In [ ]:
df["Genero_Usuario"] = df["Genero_Usuario"].replace({"?": np.nan, "O": np.nan, "o": np.nan})

print("\nConteo de géneros después de limpieza:")
print(df["Genero_Usuario"].value_counts(dropna=False))



Conteo de géneros después de limpieza:
Genero_Usuario
M      1119244
F       452935
NaN      68830
Name: count, dtype: int64


In [ ]:
before = len(df)
df = df.drop_duplicates(
    subset=["start_dt","end_dt","Bici",
            "Ciclo_Estacion_Retiro","Ciclo_EstacionArribo"]
)
after = len(df)

print(f"\nDuplicados eliminados: {before - after}")


Duplicados eliminados: 0


Edad del usuario

In [ ]:
df.loc[df["Edad_Usuario"].notna() & ~df["Edad_Usuario"].between(10,95), "Edad_Usuario"] = np.nan

age_bins   = [10, 18, 25, 35, 45, 55, 65, 95]
age_labels = ["10-17","18-24","25-34","35-44","45-54","55-64","65-95"]
df["age_bin"] = pd.cut(df["Edad_Usuario"], bins=age_bins, labels=age_labels, include_lowest=True)

print("\nEdad por grupo:")
print(df["age_bin"].value_counts().sort_index())



Edad por grupo:
age_bin
10-17      6309
18-24    270752
25-34    771782
35-44    381196
45-54    146806
55-64     52471
65-95     11577
Name: count, dtype: int64


Franja horaria de retiro

In [ ]:
hour_src = "ret_dt" if "ret_dt" in df.columns else "start_dt"
df["start_hour"] = df[hour_src].dt.hour

h_bins   = [-1, 4, 9, 13, 16, 19, 23]
h_labels = ["madrugada","pico_mañana","mediodía","tarde","pico_tarde","noche"]
df["daypart"] = pd.cut(df["start_hour"], bins=h_bins, labels=h_labels)

print("\nFranja horaria:")
print(df["daypart"].value_counts().reindex(h_labels))



Franja horaria:
daypart
madrugada        6295
pico_mañana    397121
mediodía       362603
tarde          345482
pico_tarde     349094
noche          180414
Name: count, dtype: int64


Duración del viaje

In [ ]:
# Asume df['dur_min'] ya en 0–45
bins = [0, 5, 10, 15, 30, 45]
labels = ["0-5", "5-10", "10-15", "15-30", "30-45"]
df["dur_bin"] = pd.cut(df["duration_min"], bins=bins, labels=labels, include_lowest=True)

print("\nDuración por tramo:")
print(df["dur_bin"].value_counts().sort_index())
print((df["dur_bin"].value_counts(normalize=True).sort_index()*100).round(1).astype(str) + "%")


Duración por tramo:
dur_bin
0-5      217256
5-10     483090
10-15    341338
15-30    458770
30-45    140555
Name: count, dtype: int64
dur_bin
0-5      13.2%
5-10     29.4%
10-15    20.8%
15-30    28.0%
30-45     8.6%
Name: proportion, dtype: object


In [ ]:
print("\n--- Resumen final ---")
print("Filas finales:", len(df))
print("Valores nulos por columna:\n", df.isna().sum())
print("Duración promedio (min):", round(df["duration_min"].mean(),2))


--- Resumen final ---
Filas finales: 1641009
Valores nulos por columna:
 Genero_Usuario           68830
Edad_Usuario               116
Bici                         0
Ciclo_Estacion_Retiro        0
Fecha_Retiro                 0
Hora_Retiro                  0
Ciclo_EstacionArribo         0
Fecha_Arribo                 0
Hora_Arribo                  0
Retiro_Datetime              0
Arribo_Datetime              0
Duracion_Uso                 0
start_dt                     0
end_dt                       0
duration_min                 0
age_bin                    116
start_hour                   0
daypart                      0
dur_bin                      0
dtype: int64
Duración promedio (min): 14.19
